In [1]:
import torch
import torch.nn as nn
from torchvision import models

import re
import numpy as np
import pandas as pd

#1. 데이터 가져오기 및 전처리

In [7]:
### 인장 시험 데이터 가져오기

# load tensile test result
f = open("/content/tensile_test_result.txt","r")
lines = f.readlines()
new_lines = []

# \n, 공백 삭제
for line in lines:
  new_lines.append(re.sub('[\n| ]', '', line))

# df로 만들기
new_lines_np = np.array(new_lines).reshape(-1,6)
tensile_res = [[] for _ in range(4)]
for i in range(len(new_lines_np)):
  for j in new_lines_np[i]:
    if re.match('[가-힣]',j):
      if tensile_res[i%4] == []:
        tensile_res[i%4].append(j)
    else:
      tensile_res[i%4].append(j)

tensile_res = np.array(tensile_res).reshape(4,-1)
df_tensile_res = pd.DataFrame(tensile_res[:,1:], index = tensile_res[:,0])
df_tensile_res

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
시편번호-,1-1,1-2,1-3,1-4,1-5,2-1,2-2,2-3,2-4,2-5,...,4-1,4-2,4-3,4-4,4-5,5-1,5-2,5-3,5-4,5-5
인장강도N/mm2,1527,1439,1445,1421,1429,1200,1225,1256,1211,1253,...,1004,1004,1033,1019,994,896,882,918,897,928
항복강도(0.2%offset)N/mm2,1454,1359,1381,1354,1369,1113,1128,1186,1117,1170,...,907,902,927,916,896,816,780,826,831,845
연신율(GL:50mm)%,11,13,12,13,13,14,14,14,15,15,...,19,18,16,18,19,21,21,21,21,20


In [8]:
### 이미지 데이터 가져오기

# 로컬에서 데이터 업로드 하기
from google.colab import files
imgs = files.upload()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

Saving 1-1.tif to 1-1 (1).tif
Saving 1-2.tif to 1-2 (1).tif
Saving 1-3.tif to 1-3 (1).tif
Saving 1-4.tif to 1-4 (1).tif
Saving 2-1.tif to 2-1 (1).tif
Saving 2-2.tif to 2-2 (1).tif
Saving 2-3.tif to 2-3 (1).tif
Saving 2-4.tif to 2-4 (1).tif
Saving 2-5.tif to 2-5 (1).tif
Saving 3-1.tif to 3-1 (1).tif
Saving 3-2.tif to 3-2 (1).tif
Saving 3-3.tif to 3-3 (1).tif
Saving 3-4.tif to 3-4 (1).tif
Saving 3-5.tif to 3-5 (1).tif
Saving 4-1.tif to 4-1 (1).tif
Saving 4-2.tif to 4-2 (1).tif
Saving 4-3.tif to 4-3 (1).tif
Saving 4-4.tif to 4-4 (1).tif
Saving 4-5.tif to 4-5 (1).tif
Saving 5-3.tif to 5-3 (1).tif
Saving 5-4.tif to 5-4 (1).tif
Saving 5-5.tif to 5-5 (1).tif
cpu


#2. 모델링

In [ ]:
# loading resnet101
my_model = models.resnet101(pretrained = True)

# change in and out features of fc layer
in_features = my_model.fc.in_features
out_features = 1
my_model.fc = nn.Linear(in_features, out_features)

# freeze parameters when training
for params in my_model.parameters():
  params.requires_grad = False

# unfreeze parameters of fc layer when training
for params in my_model.fc.parameters():
  params.requires_grad = True

# model to device
my_model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

#3. 모델 훈련

In [ ]:
out = my_model(inp)
lr = 0.0001 # learning rate
optimizer = torch.optim.Adam(my_model.parameters(), lr=lr)
loss_fn = nn.MSELoss().to(device)